In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import datetime

In [2]:
#Extract CSV  into DataFrame
barrelPrice_csv = "Consulta_Precio_Barril_Petroleo.csv"
barrelPrice_df = pd.read_csv(barrelPrice_csv)
print(len(barrelPrice_df))
print(barrelPrice_df.dtypes)
barrelPrice_df.head()


6335
Fecha    object
SI744    object
dtype: object


,Fecha,SI744
0,01/01/1996,N/E
1,02/01/1996,N/E
2,03/01/1996,17.40
3,04/01/1996,17.41
4,05/01/1996,17.70


In [3]:
#Clean and Transform Dataframe
#1. Change column "fecha"  into datetime format 
# datetime assumes original info has Month as its first element. Its necessary to include "dayfirst=True" so the method knows first value is the DAY.
barrelPrice_df['Fecha']=pd.to_datetime(barrelPrice_df['Fecha'], dayfirst=True)

print(barrelPrice_df.dtypes)
barrelPrice_df.head()

Fecha    datetime64[ns]
SI744            object
dtype: object


,Fecha,SI744
0,1996-01-01,N/E
1,1996-01-02,N/E
2,1996-01-03,17.40
3,1996-01-04,17.41
4,1996-01-05,17.70


In [4]:
#2. Eliminate NA values
#Identify the rows with "N/E" value (by its index value)
indexNames = barrelPrice_df[ barrelPrice_df['SI744'] == "N/E" ].index
print(indexNames)

# Delete these rows  from dataFrame, using the index value identified
barrelPrice_df.drop(indexNames , inplace=True)
print(len(barrelPrice_df))
barrelPrice_df

Int64Index([   0,    1,   16,   25,   27,   35,   43,   50,   58,   66,
            ...
            6215, 6225, 6238, 6239, 6256, 6257, 6262, 6275, 6295, 6334],
           dtype='int64', length=582)
5753


,Fecha,SI744
2,1996-01-03,17.40
3,1996-01-04,17.41
4,1996-01-05,17.70
5,1996-01-08,17.54
6,1996-01-09,17.41
...,...,...
6329,2020-04-03,20.48
6330,2020-04-06,18.66
6331,2020-04-07,16.65
6332,2020-04-08,17.89


In [5]:
#3. Change "SI744" column into float format 
barrelPrice_df['SI744'] = barrelPrice_df['SI744'].astype('float64')
print(barrelPrice_df.dtypes)
barrelPrice_df

Fecha    datetime64[ns]
SI744           float64
dtype: object


,Fecha,SI744
2,1996-01-03,17.40
3,1996-01-04,17.41
4,1996-01-05,17.70
5,1996-01-08,17.54
6,1996-01-09,17.41
...,...,...
6329,2020-04-03,20.48
6330,2020-04-06,18.66
6331,2020-04-07,16.65
6332,2020-04-08,17.89


In [6]:
# 4. Separate month and year into new columns
barrelPrice_df["month"]=barrelPrice_df.Fecha.dt.strftime('%m')
barrelPrice_df["year"]=barrelPrice_df.Fecha.dt.strftime('%Y')
print(barrelPrice_df.dtypes)
barrelPrice_df

Fecha     datetime64[ns]
SI744            float64
months            object
years             object
dtype: object


,Fecha,SI744,months,years
2,1996-01-03,17.40,01,1996
3,1996-01-04,17.41,01,1996
4,1996-01-05,17.70,01,1996
5,1996-01-08,17.54,01,1996
6,1996-01-09,17.41,01,1996
...,...,...,...,...
6329,2020-04-03,20.48,04,2020
6330,2020-04-06,18.66,04,2020
6331,2020-04-07,16.65,04,2020
6332,2020-04-08,17.89,04,2020


In [7]:
#5A. Find average barrel price per month, grouping by year and month - MEAN / reset_index  --> Final DF 292 rows (1 per month)
#Create a copy of DF
barrelPrice_df2=barrelPrice_df.copy()
#Groupby and find average value
barrelPrice_avg =barrelPrice_df2.groupby(["year","month"]).mean()
barrelPrice_avg

SI744
years months           
1996  01      16.273000
      02      16.489412
      03      18.317368
      04      19.588947
      05      17.798824
...                 ...
2019  12      54.906500
2020  01      53.897143
      02      45.386842
      03      23.259545
      04      16.697143

[292 rows x 1 columns]

In [8]:
#Create new DF by using Reset index 
barrelPrice_wAVG_df= barrelPrice_avg.reset_index()
barrelPrice_wAVG_df

,years,months,SI744
0,1996,01,16.273000
1,1996,02,16.489412
2,1996,03,18.317368
3,1996,04,19.588947
4,1996,05,17.798824
...,...,...,...
287,2019,12,54.906500
288,2020,01,53.897143
289,2020,02,45.386842
290,2020,03,23.259545


In [10]:
#Rename column 'SI744' 
barrelPrice_wAVG_df=barrelPrice_wAVG_df.rename(columns={"SI744":"barrel_avg_price"})
print(barrelPrice_wAVG_df.dtypes)
barrelPrice_wAVG_df

id                    int64
years                object
months               object
barrel_avg_price    float64
dtype: object


,id,years,months,barrel_avg_price
0,0,1996,01,16.273000
1,1,1996,02,16.489412
2,2,1996,03,18.317368
3,3,1996,04,19.588947
4,4,1996,05,17.798824
...,...,...,...,...
287,287,2019,12,54.906500
288,288,2020,01,53.897143
289,289,2020,02,45.386842
290,290,2020,03,23.259545


In [11]:
#6. Create DB Connection
connection_string = "postgres:Ichliebe1*@localhost:5432/Project2"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables
engine.table_names() 

['barrel_price', 'inflation', 'indice']

In [12]:
#7. Load DF into DB
barrelPrice_wAVG_df.to_sql(name="barrel_price", con=engine, if_exists="append", index=False)
